In [154]:
import pandas as pd
df = pd.read_csv('Data/arc.csv')
print(df.head())

          name  version                                             name.1  \
0  ArcPlatform        1     pl.arcilook.controller.JabberAccountController   
1  ArcPlatform        1  pl.arcilook.controller.settings.TaskController...   
2  ArcPlatform        1        pl.arcilook.exception.ServiceLogicException   
3  ArcPlatform        1                 pl.arcilook.service.ContactService   
4  ArcPlatform        1   pl.arcilook.exception.NotProjectManagerException   

   wmc  dit  noc  cbo  rfc  lcom  ca  ...       dam  moa       mfa       cam  \
0    7    1    0    6   19     7   0  ...  1.000000    2  0.000000  0.366667   
1   13    1    0    1   17    64   0  ...  1.000000    0  0.000000  0.461538   
2    7    4    4   48   12    19  47  ...  0.333333    0  0.866667  0.458333   
3    4    1    0    4    4     6   2  ...  0.000000    0  0.000000  1.000000   
4    4    5    0    2    8     6   1  ...  0.000000    0  1.000000  0.666667   

   ic  cbm        amc  max_cc  avg_cc  bug  
0   0

In [155]:
df = df.drop(['name', 'version', 'name.1'], axis=1)

print(df.head())

   wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  ...       dam  moa  \
0    7    1    0    6   19     7   0   6    6  0.750000  ...  1.000000    2   
1   13    1    0    1   17    64   0   1   12  0.916667  ...  1.000000    0   
2    7    4    4   48   12    19  47   1    6  0.944444  ...  0.333333    0   
3    4    1    0    4    4     6   2   2    4  2.000000  ...  0.000000    0   
4    4    5    0    2    8     6   1   1    4  2.000000  ...  0.000000    0   

        mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0  0.000000  0.366667   0    0  15.142857       3  1.1429    0  
1  0.000000  0.461538   0    0   7.461538       3  1.3846    0  
2  0.866667  0.458333   0    0   4.857143       1  0.2857    0  
3  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
4  1.000000  0.666667   0    0   4.000000       0  0.0000    0  

[5 rows x 21 columns]


In [156]:
import numpy as np

X_sample = np.array(df.drop(['bug'], axis=1), dtype='float32')
y_sample = np.array(df['bug'], dtype='uint8')

y_sample[y_sample>0] = 1

print(y_sample.shape)

(234,)


In [157]:
df = pd.read_csv('Data/xerces-1.2.csv')
df = df.drop(['name', 'version', 'name.1'], axis=1)
print(df.head())

   wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  ...       dam  moa  \
0    6    3    3    0   10     0   0   0    6  0.000000  ...  1.000000    0   
1   63    2    3   27  225   793   7  20   42  0.872080  ...  0.896552    6   
2    1    5    0    1    2     0   1   0    1  2.000000  ...  0.000000    0   
3   21    4    0    2   25   210   0   2   21  2.000000  ...  0.000000    0   
4   25    3    3   13   67    34   3  11   14  0.638889  ...  0.833333    3   

        mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0  0.850000  0.555556   1    2   7.833333       3  1.0000    0  
1  0.678947  0.152074   2   11  88.698413       6  1.6984    0  
2  1.000000  1.000000   0    0   5.000000       0  0.0000    1  
3  0.824561  0.523810   0    0   4.714286       1  0.9524    0  
4  0.684932  0.205000   2   22  43.560000      11  2.2800    2  

[5 rows x 21 columns]


In [158]:
X_target = np.array(df.drop(['bug'], axis=1), dtype='float32')
y_target = np.array(df['bug'], dtype='uint8')

y_target[y_target>0] = 1

In [159]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, random_state=13)

In [160]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)

In [161]:
from imblearn.over_sampling import SMOTE

imb = SMOTE()
X_train, y_train = imb.fit_resample(X_train, y_train)

In [162]:
from sklearn.naive_bayes import GaussianNB
from skopt import BayesSearchCV
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from AlgoCPCD import TSboostDF

X_target = sc.transform(X_target)

# param_grid = {
#             'base_estimator' : [KNeighborsClassifier()],
#             'max_features' : [0.2, 0.4, 0.6, 0.8, 1.0],
#             'base_estimator__n_neighbors' : [3,5,11,19]
#             }

# param_grid = {
#     'var_smoothing' : np.logspace(0, -9, num=300)
# }

# param_grid = {
#     'n_neighbors' : [3,5,11,19],
#     'weights' : ['uniform', 'distance'],
#     'metric' : ['euclidean', 'manhattan']
# }

# model = BayesSearchCV(estimator=BaggingClassifier(), 
#                     search_spaces=param_grid,
#                 n_iter=10,
#                 scoring='f1',
#                 cv=3)

model = BaggingClassifier(base_estimator=KNeighborsClassifier())


model.fit(X_train, y_train)

# model = TSboostDF()
# model.fit(X_train, y_train, X_target)

BaggingClassifier(base_estimator=KNeighborsClassifier())

In [163]:
from sklearn.metrics import f1_score, matthews_corrcoef
X_test = sc.transform(X_test)
y_pred = model.predict(X_test)
print(f1_score(y_test, y_pred), matthews_corrcoef(y_test, y_pred))

0.125 0.031227829014591122


In [164]:
from sklearn.metrics import f1_score, matthews_corrcoef
y_pred = model.predict(X_target)
print(f1_score(y_target, y_pred), matthews_corrcoef(y_target, y_pred))

0.1951219512195122 0.01502909632169254
